In [1]:
import pandas as pd
import geopandas as gpd
import folium

pd.set_option('display.max_rows', 100)

## Solar Readiness Assessment

The solar readiness of each building represents the ability to host solar PV systems in their current condition. Buildings must be: 

1. 10,000 square feet or more. 
2. Roof that's 10 years old or newer. 
3. Be in good structural condition. 

Read more at: https://www.nyc.gov/assets/dcas/downloads/pdf/energy/reportsandpublication/local24_2022.pdf

In [2]:
schools = pd.read_excel("../data/raw_data/DOE/School Locations/LCGMS/LCGMS_SchoolData_20251020_2055.xlsx")
solar_readiness = pd.read_csv("../data/raw_data/DCAS/solar_readiness/City_of_New_York_Municipal_Solar-Readiness_Assessment_(Local_Law_24_of_2016)_data_20251020.csv")

In [3]:
solar_readiness.query("Agency == 'DOE'").query("Site == 'R036'").T

,256,511,2352,2604
City Council District,51,51,51,51
Agency,DOE,DOE,DOE,DOE
Site,R036,R036,R036,R036
Address,255 Ionia Ave,255 Ionia Ave,255 Ionia Ave,255 Ionia Ave
Borough,Staten Island,Staten Island,Staten Island,Staten Island
Environmental Justice Area,No,No,�,�
Installation Date,NaN,NaN,NaN,NaN
Installed or Estimated Capacity,285,285,285.00 kW,285.00 kW
Percentage of Max Peak Demand,84.40%,NaN,0.84,"313,500 kWh"
Estimated Annual Production,313500,313500,"313,500 kWh",NaN


In [4]:
# sizeable chunk (4.2k from solar readiness, 1.9k from schools, only 1.5k left. Since we only survey buildings above 100k sqft, this seems not unreasonable.)
# we use building site, I feel like this isn't perfect, b/c maybe two buildings share the same site, but for now I can't think of a better way. 
df = solar_readiness.merge(schools, left_on="Site", right_on="Location Code", how='inner')
print(len(df), len(solar_readiness), len(schools))
print(solar_readiness.groupby("Site").size().describe())
print(schools.groupby("Location Code").size().describe())

1557 4268 1901
count    2255.000000
mean        1.892683
std         0.527012
min         1.000000
25%         2.000000
50%         2.000000
75%         2.000000
max        18.000000
dtype: float64
count    1901.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
dtype: float64


In [5]:
gdf = gpd.GeoDataFrame(df[[
    # solar readiness columns
    "Site", 
    "Address", 
    "Borough", 
    "Status", 
    "Solar-Readiness Assessment",
    "Latitude", 
    "Longitude", 
    "BIN", 
    "BBL", 
    "Total Gross Square Footage",
    "Roof Condition",
    "Roof Age", 
    "Estimated Annual Emissions Reduction", 
    "Estimated Social Cost of Carbon Value", 
    "Estimated Annual Energy Savings",

    # school points columns
    "Location Name",
    "BEDS Number",
]].assign(Status=lambda df: df.Status.str.replace("-", " ")).dropna(subset=["Latitude", "Longitude"])) # post processing
gdf.to_csv("../data/solar_readiness/solar_readiness_per_site.csv")

In [6]:
!rm -f ../data/solar_readiness/solar_readiness_per_site.geojson
gdf.to_file("../data/solar_readiness/solar_readiness_per_site.geojson", driver="GeoJson")

In [7]:
# Weirdly, Status == 'Solar Ready' is not the same as Solar-Readiness Assessment 'Yes'
# Maybe worth digging into if we're specifically trying to install solar panels.
print(gdf.assign(Status=lambda df: df.Status.str.replace("-", " ")).groupby(['Status', 'Solar-Readiness Assessment']).size())
gdf.query("Status == 'Solar Ready' and `Solar-Readiness Assessment` == 'No'").head().T

Status           Solar-Readiness Assessment
Not Solar Ready  No                            1126
                 Yes                             41
Solar Ready      No                             130
                 Yes                             24
dtype: int64


,112,113,114,115,116
Site,M520,M114,M183,M191,M485
Address,411 Pearl St,331 East 91st St,419 East 66 St,210 West 61 St,100 Amsterdam Ave
Borough,Manhattan,Manhattan,Manhattan,Manhattan,Manhattan
Status,Solar Ready,Solar Ready,Solar Ready,Solar Ready,Solar Ready
Solar-Readiness Assessment,No,No,No,No,No
Latitude,40.711442,40.780706,40.763736,40.771925,40.773629
Longitude,-74.000851,-73.948553,-73.95815,-73.987328,-73.985273
BIN,1001388.0,1081267.0,1045569.0,1030320.0,1030341.0
BBL,1001130100.0,1015540032.0,1014610007.0,1011520029.0,1011560030.0
Total Gross Square Footage,"300,000 GSF","48,000 GSF","58,800 GSF","75,000 GSF","388,000 GSF"


In [8]:
m = folium.Map(location=[40.7128, -74.0060], zoom_start=11)  # pick a suitable center

for readiness, sub in gdf.groupby("Solar-Readiness Assessment"):
    fg = folium.FeatureGroup(name=f"{readiness} ({len(sub)})", show=True)
    for _, r in sub.iterrows():
        folium.CircleMarker(
            location=[r["Latitude"], r["Longitude"]],
            radius=2,
            color="red" if readiness == 'No' else "green",
            weight=1,
            fill=True,
            fill_opacity=0.3 if readiness == 'No' else 0.8,
            popup=folium.Popup(f"<b>{r['Location Name']}</b><br>Roof Age: {r['Roof Age']}<br>Roof Condition: {r['Roof Condition']}<br>Est. Emmissions Reduction: {r['Estimated Annual Emissions Reduction']}", max_width=250),
            tooltip=r["Location Name"],
        ).add_to(fg)
    fg.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)